In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import IsolationForest
set_config(display='diagram') # Useful for display the pipeline
print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.4
Sklearn  0.24.2


In [2]:
df = pd.read_csv('Building_Permits.csv', low_memory=False)  

In [3]:
df.describe()

,Permit Type,Street Number,Unit,Number of Existing Stories,Number of Proposed Stories,Estimated Cost,Revised Cost,Existing Units,Proposed Units,Plansets,Existing Construction Type,Proposed Construction Type,Supervisor District,Zipcode,Record ID
count,198900.000000,198900.000000,29479.000000,156116.000000,156032.000000,1.608340e+05,1.928340e+05,147362.000000,147989.000000,161591.000000,155534.000000,155738.000000,197183.000000,197184.000000,1.989000e+05
mean,7.522323,1121.728944,78.517182,5.705773,5.745043,1.689554e+05,1.328562e+05,15.666164,16.510950,1.274650,4.072878,4.089529,5.538403,94115.500558,1.162048e+12
std,1.457451,1135.768948,326.981324,8.613455,8.613284,3.630386e+06,3.584903e+06,74.476321,75.220444,22.407345,1.585756,1.578766,2.887041,9.270131,4.918215e+11
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,94102.000000,1.293532e+10
25%,8.000000,235.000000,0.000000,2.000000,2.000000,3.300000e+03,1.000000e+00,1.000000,1.000000,0.000000,3.000000,3.000000,3.000000,94109.000000,1.308567e+12
50%,8.000000,710.000000,0.000000,3.000000,3.000000,1.100000e+04,7.000000e+03,1.000000,2.000000,2.000000,5.000000,5.000000,6.000000,94114.000000,1.371840e+12
75%,8.000000,1700.000000,1.000000,4.000000,4.000000,3.500000e+04,2.870750e+04,4.000000,4.000000,2.000000,5.000000,5.000000,8.000000,94122.000000,1.435000e+12
max,8.000000,8400.000000,6004.000000,78.000000,78.000000,5.379586e+08,7.805000e+08,1907.000000,1911.000000,9000.000000,5.000000,5.000000,11.000000,94158.000000,1.498342e+12


In [4]:
df.head(15)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992
5,201706149344,8,otc alterations permit,06/14/2017,4105,009,800,NaN,Indiana,St,...,1.0,constr type 1,1.0,constr type 1,NaN,10.0,Potrero Hill,94107.0,"(37.75922331346539, -122.39170402628598)",1466911170855
6,201706300814,8,otc alterations permit,06/30/2017,1739,020,1291,NaN,11th,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,5.0,Inner Sunset,94122.0,"(37.764145640138565, -122.46875112470363)",1468970403692
7,M803667,8,otc alterations permit,06/30/2017,4789,014,1465,NaN,Revere,Av,...,NaN,NaN,NaN,NaN,NaN,10.0,Bayview Hunters Point,94124.0,"(37.73005099023611, -122.38784938916618)",1469035175050
8,M804227,8,otc alterations permit,07/05/2017,1212,054,2094,NaN,Fell,St,...,NaN,NaN,NaN,NaN,NaN,5.0,Lone Mountain/USF,94117.0,"(37.772393498502595, -122.45231466824669)",1469198253772
9,M804767,8,otc alterations permit,07/06/2017,1259,016,89,NaN,Alpine,Tr,...,NaN,NaN,NaN,NaN,NaN,8.0,Haight Ashbury,94117.0,"(37.7691724293766, -122.43734859051908)",146932394171


In [5]:
df.columns

Index(['Permit Number', 'Permit Type', 'Permit Type Definition',
       'Permit Creation Date', 'Block', 'Lot', 'Street Number',
       'Street Number Suffix', 'Street Name', 'Street Suffix', 'Unit',
       'Unit Suffix', 'Description', 'Current Status', 'Current Status Date',
       'Filed Date', 'Issued Date', 'Completed Date',
       'First Construction Document Date', 'Structural Notification',
       'Number of Existing Stories', 'Number of Proposed Stories',
       'Voluntary Soft-Story Retrofit', 'Fire Only Permit',
       'Permit Expiration Date', 'Estimated Cost', 'Revised Cost',
       'Existing Use', 'Existing Units', 'Proposed Use', 'Proposed Units',
       'Plansets', 'TIDF Compliance', 'Existing Construction Type',
       'Existing Construction Type Description', 'Proposed Construction Type',
       'Proposed Construction Type Description', 'Site Permit',
       'Supervisor District', 'Neighborhoods - Analysis Boundaries', 'Zipcode',
       'Location', 'Record ID'],
     

In [6]:
round(df.isnull().sum() * 100 / len(df),2)

Permit Number                               0.00
Permit Type                                 0.00
Permit Type Definition                      0.00
Permit Creation Date                        0.00
Block                                       0.00
Lot                                         0.00
Street Number                               0.00
Street Number Suffix                       98.89
Street Name                                 0.00
Street Suffix                               1.39
Unit                                       85.18
Unit Suffix                                99.01
Description                                 0.15
Current Status                              0.00
Current Status Date                         0.00
Filed Date                                  0.00
Issued Date                                 7.51
Completed Date                             51.14
First Construction Document Date            7.51
Structural Notification                    96.52
Number of Existing S

# Drop columns
Street Number Suffix                       98.89
Proposed Construction Type Description     21.70
Unit                                       85.18
Unit Suffix                                99.01
Completed Date                             51.14
Structural Notification                    96.52
Voluntary Soft-Story Retrofit              99.98
Existing Construction Type Description     21.80
Site Permit                                97.31
TIDF Compliance                           100.00
Fire Only Permit                          90.53

In [7]:
df=df.drop(['Street Number Suffix','Proposed Construction Type Description','Unit','Unit Suffix' ,'Completed Date','Structural Notification','Voluntary Soft-Story Retrofit','Existing Construction Type Description','Site Permit','TIDF Compliance','Fire Only Permit'],axis=1)


In [8]:
round(df.isnull().sum() * 100 / len(df),2)

Permit Number                           0.00
Permit Type                             0.00
Permit Type Definition                  0.00
Permit Creation Date                    0.00
Block                                   0.00
Lot                                     0.00
Street Number                           0.00
Street Name                             0.00
Street Suffix                           1.39
Description                             0.15
Current Status                          0.00
Current Status Date                     0.00
Filed Date                              0.00
Issued Date                             7.51
First Construction Document Date        7.51
Number of Existing Stories             21.51
Number of Proposed Stories             21.55
Permit Expiration Date                 26.08
Estimated Cost                         19.14
Revised Cost                            3.05
Existing Use                           20.67
Existing Units                         25.91
Proposed U

In [9]:
print(f"{df['Number of Existing Stories'].head()} ")          
print(f"{df['Number of Proposed Stories'].head()}   ")         
print(f"{df['Permit Expiration Date'].head()}" )               
print(f"{df['Estimated Cost'].head()}")                       

0    6.0
1    7.0
2    6.0
3    2.0
4    3.0
Name: Number of Existing Stories, dtype: float64 
0    NaN
1    NaN
2    6.0
3    2.0
4    NaN
Name: Number of Proposed Stories, dtype: float64   
0    11/03/2016
1    12/03/2017
2           NaN
3    07/13/2018
4    12/01/2018
Name: Permit Expiration Date, dtype: object
0      4000.0
1         1.0
2     20000.0
3      2000.0
4    100000.0
Name: Estimated Cost, dtype: float64


In [10]:
print(f"{df['Existing Use'].head()}")
print(f"{df['Existing Units'].head()}")
print(f"{df['Proposed Use'].head()} ")
print(f"{df['Proposed Units'].head()}")
print(f"{df['Plansets'].head()}")
print(f"{df['Existing Construction Type'].head()}")
print(f"{df['Proposed Construction Type'].head()}")                            
                        
                              
           
          

0    tourist hotel/motel
1    tourist hotel/motel
2           retail sales
3      1 family dwelling
4           retail sales
Name: Existing Use, dtype: object
0    143.0
1      NaN
2     39.0
3      1.0
4      NaN
Name: Existing Units, dtype: float64
0                  NaN
1                  NaN
2         retail sales
3    1 family dwelling
4                  NaN
Name: Proposed Use, dtype: object 
0     NaN
1     NaN
2    39.0
3     1.0
4     NaN
Name: Proposed Units, dtype: float64
0    2.0
1    2.0
2    2.0
3    2.0
4    2.0
Name: Plansets, dtype: float64
0    3.0
1    3.0
2    1.0
3    5.0
4    3.0
Name: Existing Construction Type, dtype: float64
0    NaN
1    NaN
2    1.0
3    5.0
4    NaN
Name: Proposed Construction Type, dtype: float64


In [21]:
car_columns=['Proposed Use','Existing Use','Permit Expiration Date']
num_columns=['Existing Units','Proposed Construction Type','Existing Construction Type','Plansets','Proposed Units','Estimated Cost','Number of Proposed Stories','Number of Existing Stories']

In [22]:
df[num_columns]

,Existing Units,Proposed Construction Type,Existing Construction Type,Plansets,Proposed Units,Estimated Cost,Number of Proposed Stories,Number of Existing Stories
0,143.0,NaN,3.0,2.0,NaN,4000.0,NaN,6.0
1,NaN,NaN,3.0,2.0,NaN,1.0,NaN,7.0
2,39.0,1.0,1.0,2.0,39.0,20000.0,6.0,6.0
3,1.0,5.0,5.0,2.0,1.0,2000.0,2.0,2.0
4,NaN,NaN,3.0,2.0,NaN,100000.0,NaN,3.0
...,...,...,...,...,...,...,...,...
198895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198896,4.0,5.0,5.0,2.0,4.0,5000.0,4.0,4.0
198897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# X = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
# y = df["Survived"] # 0 = No, 1 = Yes

# x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin'])

In [13]:
df.columns

Index(['Permit Number', 'Permit Type', 'Permit Type Definition',
       'Permit Creation Date', 'Block', 'Lot', 'Street Number', 'Street Name',
       'Street Suffix', 'Description', 'Current Status', 'Current Status Date',
       'Filed Date', 'Issued Date', 'First Construction Document Date',
       'Number of Existing Stories', 'Number of Proposed Stories',
       'Permit Expiration Date', 'Estimated Cost', 'Revised Cost',
       'Existing Use', 'Existing Units', 'Proposed Use', 'Proposed Units',
       'Plansets', 'Existing Construction Type', 'Proposed Construction Type',
       'Supervisor District', 'Neighborhoods - Analysis Boundaries', 'Zipcode',
       'Location', 'Record ID'],
      dtype='object')

In [14]:
df.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Street Suffix,Description,...,Proposed Use,Proposed Units,Plansets,Existing Construction Type,Proposed Construction Type,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,Ellis,St,"ground fl facade: to erect illuminated, electr...",...,NaN,NaN,2.0,3.0,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,Geary,St,remove (e) awning and associated signs.,...,NaN,NaN,2.0,3.0,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,Pacific,Av,installation of separating wall,...,retail sales,39.0,2.0,1.0,1.0,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,Pacific,Av,repair dryrot & stucco at front of bldg.,...,1 family dwelling,1.0,2.0,5.0,5.0,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,St,demolish retail/office/commercial 3-story buil...,...,NaN,NaN,2.0,3.0,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


In [34]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

num_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('scalar',StandardScaler()),('normalize',PowerTransformer())
])

cat_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(missing_values=np.nan, strategy='most_frequent', fill_value=None, verbose=0, copy=True, add_indicator=False)),
  ('onehot', preprocessing.OneHotEncoder(handle_unknown='error'))
])

prepro = compose.ColumnTransformer(transformers=[
    ('num', num_treeModels, num_columns),
    ('cat', cat_treeModels, car_columns),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars


classifiers = {
  "Decision Tree": DecisionTreeClassifier(random_state=4),
  "Extra Trees":ExtraTreesClassifier(random_state=4),
  "Random Forest":RandomForestClassifier(bootstrap =True,max_depth =7,max_features= 'auto',min_samples_leaf=4,min_samples_split= 10,n_estimators= 200),
  "AdaBoost":AdaBoostClassifier(n_estimators=200, random_state=10,learning_rate=0.1),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":HistGradientBoostingClassifier(random_state=1),
  "XGBoost":XGBClassifier(random_state=1),
  "LightGBM":LGBMClassifier(random_state=1),
  "CatBoost":CatBoostClassifier(random_state=2)
}

classifiers = {name: pipeline.make_pipeline(prepro,model) for name, model in classifiers.items()}

In [35]:
im= impute.SimpleImputer(missing_values=np.nan, strategy='mean')


In [37]:
f=im.fit(df[num_columns])

In [31]:
df[num_columns]

,Existing Units,Proposed Construction Type,Existing Construction Type,Plansets,Proposed Units,Estimated Cost,Number of Proposed Stories,Number of Existing Stories
0,143.0,NaN,3.0,2.0,NaN,4000.0,NaN,6.0
1,NaN,NaN,3.0,2.0,NaN,1.0,NaN,7.0
2,39.0,1.0,1.0,2.0,39.0,20000.0,6.0,6.0
3,1.0,5.0,5.0,2.0,1.0,2000.0,2.0,2.0
4,NaN,NaN,3.0,2.0,NaN,100000.0,NaN,3.0
...,...,...,...,...,...,...,...,...
198895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198896,4.0,5.0,5.0,2.0,4.0,5000.0,4.0,4.0
198897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    X_train, Y_train,
    test_size=0.2,
    stratify = Y_train,   
    random_state=3 )

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_val)
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 